In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 09: Text + multimodal embedding generation and vector search in BigQuery




## Overview

### Objectives

In this tutorial, you'll build a [**semantic search**](https://cloud.google.com/discover/what-is-semantic-search) utility for a fictional retail store chain. You will use [BigQuery](https://cloud.google.com/bigquery/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/docs), to go beyond simple keyword matching by generating [**vector embeddings**](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings) for product text descriptions _and_ for images. This allows customers to find products based on conceptual meaning, not just keywords, creating a more intuitive shopping experience.

By the end, you will know how to:

* **Generate embeddings (numerical representations)** for text and image data.

* **Perform text-to-text vector search** to find products with similar text descriptions

* **Perform text-to-image vector search** to find product images that visually match a text query


All within BigQuery!


The module also features the new automated embedding generation capability that is in public preview (as of Dec 8, 2025)

---

## 1. Setup

### 1.1. Set your project ID

In [ ]:
PROJECT_ID_LIST=!gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID=PROJECT_ID_LIST[0]
PROJECT_NBR_LIST=!gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NBR=PROJECT_NBR_LIST[0]
LOCATION="us-central1"
DATA_DS="rscw_oltp_stg_ds"
MULTIMODAL_STAGE_DS="rscw_multimodal_ai_stg_ds"
MULTIMODAL_STAGE_BUCKET=f"rscw_multimodal_ai_stg_bucket-{PROJECT_NBR}"


---

### 1.2. BigQuery Cloud resource connection

We will reuse the Bigquery Cloud resource connection from Module 08 called `us-central1.rscw_cloud_resource_connection_usc`

---

### 1.3. Set permissions for Service Account

The resource connection service account requires certain project-level permissions to interact with Vertex AI and Google Cloud Storage.

First, you need to find the email address associated with your new connection's service account.

In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us-central1.rscw_cloud_resource_connection_usc | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

Next you'll assign roles that give ther service account permission to be a "BigQuery Connection User," an "AI Platform User" (so it can access Gemini models in Vertex AI), and a "Storage Object Viewer" (so it can access images in Cloud Storage).

*A 60 second pause is added to allow the new IAM permissions to propagate throughout Google Cloud.*

In [ ]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/storage.objectViewer'

time.sleep(60)

---

### 1.4. Create a Cloud Storage Bucket for images

In [ ]:
!gcloud storage buckets create gs://$MULTIMODAL_STAGE_BUCKET --project=$PROJECT_ID --location=$LOCATION

---

### 1.5. Create helper functions to generate product images to store to Google Cloud Storage

These are helpful utility functions that you'll use later in the tutorial that generate product images.

#### 1.5.1. Install requisite libraries

This is a one-time activity for the lab

In [ ]:
%pip install --upgrade --quiet google-genai google-cloud-storage google-cloud-aiplatform reportlab

#### 1.5.2. Let's try out the **image generation** for 1 product with `gemini-*-flash-image (Nano Banana)`

In [ ]:
from io import BytesIO
from IPython.display import Image, Markdown, display
from PIL import Image as PIL_Image
from google import genai
from google.genai.types import FinishReason, GenerateContentConfig, ImageConfig, Part
from google.cloud import storage  # Import Storage library
import matplotlib.image as img
import matplotlib.pyplot as plt
import requests

# --- Configuration ---
bucket_nm = MULTIMODAL_STAGE_BUCKET
sample_product_id = "ba9a847a2c023953743ba9e5ef6c88c6"
sample_product_nm = "Portable Fabric Steamer"
output_folder="generated_images"
output_filename = f"{sample_product_id}-1.png"

sample_product_description = """## Smooth & Ready, Anywhere You Go: The Portable Fabric Steamer
... (truncated for brevity, uses your full description) ...
"""

sample_product_prompt = f"Generate a high-resolution image for the product {sample_product_nm}, having the description - {sample_product_description} for a customer audience. Make the image compelling to purchase"

# --- Generation Client ---
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID = "gemini-2.5-flash-image"

print(f"Generating image for {sample_product_nm}...")

response = client.models.generate_content(
    model=MODEL_ID,
    contents=sample_product_prompt,
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="9:16",
        ),
        candidate_count=1,
    ),
)

# --- Error Handling ---
if response.candidates[0].finish_reason != FinishReason.STOP:
    reason = response.candidates[0].finish_reason
    raise ValueError(f"Prompt Content Error: {reason}")

# --- Processing & Uploading ---
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))

    if part.inline_data:
        # 1. Display in Notebook
        print("Rendering image in Colab...")
        display(Image(data=part.inline_data.data, width=500))

        # 2. Upload to Google Cloud Storage
        print(f"Persisting to gs://{bucket_nm}/{output_folder}/{output_filename}...")

        try:
            # Initialize Storage Client
            storage_client = storage.Client(project=PROJECT_ID)
            bucket = storage_client.bucket(bucket_nm)
            blob = bucket.blob(f"{output_folder}/{output_filename}")

            # Upload using BytesIO to convert bytes to file-like object
            blob.upload_from_file(
                BytesIO(part.inline_data.data),
                content_type=part.inline_data.mime_type  # Dynamically gets 'image/png' or 'image/jpeg'
            )
            print("✅ Upload successful.")

        except Exception as e:
            print(f"❌ Failed to upload to GCS: {e}")

#### 1.5.2. Lets create helper function to generate product images

In [ ]:
from io import BytesIO
from IPython.display import Image, Markdown, display
from PIL import Image as PIL_Image
from google import genai
from google.genai.types import FinishReason, GenerateContentConfig, ImageConfig, Part
from google.cloud import storage  # Import Storage library
import matplotlib.image as img
import matplotlib.pyplot as plt
import requests
import base64 # Import base64 module


# --- Generation Client ---
genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID = "gemini-2.5-flash-image"

# Initialize Google Cloud Storage client
storage_client = storage.Client(project=PROJECT_ID)



# Generate product image
def generate_persist_product_image(genai_client,storage_client, model_flavor, product_id: str, prompt: str, destination_bucket_nm: str, output_folder_nm: str):
    """
    Generates an image using the model flavor specified, based on a given prompt.
    Returns the image data as bytes.
    """

    output_file_nm = f"{output_folder_nm}/{product_id}.png"
    try:
      response = genai_client.models.generate_content(
        model=model_flavor,
        contents=prompt,
        config=GenerateContentConfig(
            response_modalities=["IMAGE"],
            image_config=ImageConfig(
                aspect_ratio="9:16",
            ),
            candidate_count=1,
        ),
      )
    except Exception as e:
      print(f"Failed to upload to GCS: {e}")

    # Check for errors if an image is not generated
    if response.candidates[0].finish_reason != FinishReason.STOP:
      reason = response.candidates[0].finish_reason
      raise ValueError(f"Prompt Content Error: {reason}")


    # Processing & uploading
    for part in response.candidates[0].content.parts:
      if part.text:
          display(Markdown(part.text))

      if part.inline_data:

          # Upload to Google Cloud Storage
          print(f"Persisting to gs://{destination_bucket_nm}/{output_file_nm}...")

          try:
              # Initialize Storage Client
              bucket = storage_client.bucket(bucket_nm)
              blob = bucket.blob(output_file_nm)

              # Upload using BytesIO to convert bytes to file-like object
              blob.upload_from_file(
                  BytesIO(part.inline_data.data),
                  content_type=part.inline_data.mime_type  # Dynamically gets 'image/png' or 'image/jpeg'
              )
              print("Generation and upload successful.")

          except Exception as e:
              print(f"Failed to upload to GCS: {e}")



# Function to be invoked to generate images
def generate_and_persist_image_for_product(genai_client,
                               storage_client,
                               model_flavor: str,
                               product_id: str,
                               product_nm: str,
                               product_description: str,
                               destination_bucket_nm: str,
                               output_folder_nm: str) -> str:

    prompt = f"Generate a high-resolution image for the product {product_nm}, having the description - {product_description} for a customer audience. Make the image compelling to purchase"
    try:
        generate_persist_product_image(genai_client,storage_client,model_flavor,product_id,prompt,destination_bucket_nm,output_folder_nm)

        return "Generated image successfully & persisted it to GCS"

    except Exception as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"

#### 1.5.3. Lets try out the function

In [ ]:
# Try out the function created

model_flavor = "gemini-2.5-flash-image"
image_folder_nm = "generated_images"
sample_product_id = "ba9a847a2c023953743ba9e5ef6c88c6"
sample_product_nm = "Portable Fabric Steamer"
sample_product_description = """## Smooth & Ready, Anywhere You Go: The Portable Fabric Steamer
Say goodbye to stubborn wrinkles and hello to effortlessly fresh, crisp garments – no matter where you are! The Portable Fabric Steamer is your ultimate companion for maintaining a polished look, from quick touch-ups at home to perfecting your outfit on the go.

**Why You'll Love It:**

*   **Ultra-Compact & Lightweight:** Designed for life on the move, its sleek, lightweight build fits easily into any suitcase, backpack, or even a desk drawer. Never travel with wrinkled clothes again!
*   **Rapid Heat-Up:** No more waiting around! Get powerful, continuous steam in **just 30 seconds** for instant wrinkle removal when you need it most.
*   **Powerful & Effective:** Don't let its size fool you. This steamer delivers a strong, steady steam flow that penetrates fabric fibers, smoothing out creases and refreshing garments without damaging delicate materials.
*   **Gentle on All Fabrics:** From delicate silk and satin to sturdy cotton, linen, wool, and synthetics – safely de-wrinkle and revitalize all your favorite garments, drapes, and even upholstery.
*   **Easy to Use:** Simply fill the water tank, plug it in, and press a button. Its ergonomic design makes steaming comfortable and hassle-free, eliminating the need for an ironing board.
*   **Beyond Wrinkles:** More than just de-wrinkling, the hot steam helps to sanitize and deodorize fabrics, eliminating odors and refreshing garments between washes.
*   **Safety First:** Equipped with an automatic shut-off feature for peace of mind, ensuring safe operation every time.

**Perfect for:**

*   Travelers and business trips
*   Quick morning touch-ups
*   Delicate fabrics that can't be ironed
*   Students and small apartments
*   Refreshing curtains, upholstery, and bedding

Invest in convenience, confidence, and impeccable style. The Portable Fabric Steamer is a must-have for anyone who values looking sharp without the fuss.

**Grab yours today and experience the freedom of a wrinkle-free life!**"""


generate_and_persist_image_for_product(genai_client,
                               storage_client,
                               model_flavor,
                               sample_product_id,
                               sample_product_nm,
                               sample_product_description,
                               MULTIMODAL_STAGE_BUCKET,
                               image_folder_nm)

### 1.6. Create helper functions to generate product user manuals

In [ ]:
import io
import vertexai
from vertexai.generative_models import GenerativeModel, Part as vertex_genai_models_part
from google.cloud import storage

# ReportLab imports for PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch


def get_image_data_from_gcs(gcs_uri, storage_client):
    """
    Downloads an image from GCS into a BytesIO object for PDF embedding.
    Returns: (BytesIO object, bucket_name, blob_name)
    """


    print("Inside get_image_data_from_gcs")

    # Parse gs://bucket/blob
    parts = gcs_uri.replace("gs://", "").split("/", 1)
    bucket_name = parts[0]
    blob_name = parts[1]


    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    img_buffer = io.BytesIO()
    blob.download_to_file(img_buffer)
    img_buffer.seek(0) # Reset pointer to start
    return img_buffer

def generate_product_user_manual_content(product_nm, product_description, product_image_part, model_flavor):
    """
    Asks Gemini to write the manual formatted with simple HTML tags
    that ReportLab can render (<b>, <br/>, etc).
    """

    model = GenerativeModel(model_flavor)

    # We ask for HTML-like tags because ReportLab's Paragraph engine supports them.
    # Note: We strictly forbid Markdown code fences.
    text_prompt = f"""
    You are a technical writer. Write a user manual for the product shown in the image.

    Product Name: {product_nm}
    Product Description: {product_description}

    INSTRUCTIONS:
    1. Analyze the image for features (buttons, lights, ports).
    2. Write the manual using specific XML/HTML tags for formatting:
       - Wrap Section Headers in <b> tags (e.g., <b>Getting Started</b>).
       - Use <br/> for line breaks.
       - Do NOT use Markdown (no #, no **).
       - Do NOT use <html> or <body> tags.

    Structure:
    - <b>Introduction</b>
    - <b>Visual Tour</b> (Describe the features seen in the image, as well as inferred from the product description)
    - <b>Operation Instructions</b>
    - <b>Safety</b>
    """

    response = model.generate_content(
        [text_prompt, product_image_part],
        generation_config={"temperature": 0.2}
    )

    return response.text

def create_pdf(product_nm, product_user_manual_text, product_image_buffer):
    """
    Generates a PDF file in memory.
    """

    pdf_buffer = io.BytesIO()
    doc = SimpleDocTemplate(pdf_buffer, pagesize=letter)

    # Styles
    styles = getSampleStyleSheet()
    title_style = styles['Title']
    body_style = styles['BodyText']
    body_style.spaceAfter = 12  # Add space between paragraphs

    story = []

    # 1. Add Product Name (Title)
    story.append(Paragraph(f"User Manual: {product_nm}", title_style))
    story.append(Spacer(1, 0.2 * inch))

    # 2. Add Product Image
    # Resize image to fit width (approx 5 inches wide) while keeping aspect ratio
    try:
        img = RLImage(product_image_buffer)
        img_width = 5 * inch
        aspect = img.imageHeight / float(img.imageWidth)
        img.drawHeight = img_width * aspect
        img.drawWidth = img_width
        story.append(img)
        story.append(Spacer(1, 0.3 * inch))
    except Exception as e:
        print(f"Could not embed image: {e}")

    # 3. Add Gemini Generated Text
    # We replace newlines with <br/> just in case Gemini missed some,
    # ensuring the PDF renders separate lines.
    formatted_text = product_user_manual_text.replace("\n", "<br/>")

    # Create the paragraph
    p = Paragraph(formatted_text, body_style)
    story.append(p)

    # Build PDF
    doc.build(story)
    pdf_buffer.seek(0)
    return pdf_buffer

def save_pdf_to_gcs(bucket_name, file_name, pdf_buffer,storage_client):
    """
    Uploads the PDF bytes to GCS.
    """

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    blob.upload_from_file(pdf_buffer, content_type="application/pdf")
    print(f"Success! PDF saved to: gs://{bucket_name}/{file_name}")


def generate_and_persist_user_manual_pdf(product_id,product_nm, product_description,product_image_uri,model_flavor,image_folder_nm, storage_client):
    try:
      # 1. Get the raw product image
      product_image_data = get_image_data_from_gcs(product_image_uri,storage_client)

      # 2. Gemini needs the image part, get the same
      product_image_part = vertex_genai_models_part.from_data(product_image_data.getvalue(), mime_type="image/png")

      # 3. Generating user manual content with image, product name, and product description as input")
      product_user_manual_content = generate_product_user_manual_content(product_nm, product_description, product_image_part,model_flavor)

      # 4. Building PDF off of the generated user manual content
      # Reset image buffer pointer again for PDF library reading
      product_image_data.seek(0)
      product_user_manual_pdf_file = create_pdf(product_nm, product_user_manual_content, product_image_data)

      # 5. Uploading PDF to GCS
      save_pdf_to_gcs(MULTIMODAL_STAGE_BUCKET, f"generated_user_manuals/{product_id}.pdf", product_user_manual_pdf_file,storage_client)

    except Exception as e:
      print(f"Could not create user manual PDF: {e}")

In [ ]:
# Lets give it a try
user_manual_generation_model_flavor = "gemini-2.5-pro"
image_folder_nm = "generated_images"
sample_product_id = "ba9a847a2c023953743ba9e5ef6c88c6"
sample_product_nm = "Portable Fabric Steamer"
sample_product_description = """## Smooth & Ready, Anywhere You Go: The Portable Fabric Steamer
Say goodbye to stubborn wrinkles and hello to effortlessly fresh, crisp garments – no matter where you are! The Portable Fabric Steamer is your ultimate companion for maintaining a polished look, from quick touch-ups at home to perfecting your outfit on the go.

**Why You'll Love It:**

*   **Ultra-Compact & Lightweight:** Designed for life on the move, its sleek, lightweight build fits easily into any suitcase, backpack, or even a desk drawer. Never travel with wrinkled clothes again!
*   **Rapid Heat-Up:** No more waiting around! Get powerful, continuous steam in **just 30 seconds** for instant wrinkle removal when you need it most.
*   **Powerful & Effective:** Don't let its size fool you. This steamer delivers a strong, steady steam flow that penetrates fabric fibers, smoothing out creases and refreshing garments without damaging delicate materials.
*   **Gentle on All Fabrics:** From delicate silk and satin to sturdy cotton, linen, wool, and synthetics – safely de-wrinkle and revitalize all your favorite garments, drapes, and even upholstery.
*   **Easy to Use:** Simply fill the water tank, plug it in, and press a button. Its ergonomic design makes steaming comfortable and hassle-free, eliminating the need for an ironing board.
*   **Beyond Wrinkles:** More than just de-wrinkling, the hot steam helps to sanitize and deodorize fabrics, eliminating odors and refreshing garments between washes.
*   **Safety First:** Equipped with an automatic shut-off feature for peace of mind, ensuring safe operation every time.

**Perfect for:**

*   Travelers and business trips
*   Quick morning touch-ups
*   Delicate fabrics that can't be ironed
*   Students and small apartments
*   Refreshing curtains, upholstery, and bedding

Invest in convenience, confidence, and impeccable style. The Portable Fabric Steamer is a must-have for anyone who values looking sharp without the fuss.

**Grab yours today and experience the freedom of a wrinkle-free life!**"""
sample_product_image_gcs_uri=f"gs://{MULTIMODAL_STAGE_BUCKET}/generated_images/{sample_product_id}.png"

genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
storage_client = storage.Client(project=PROJECT_ID)

generate_and_persist_user_manual_pdf(sample_product_id,
                                     sample_product_nm,
                                     sample_product_description,
                                     sample_product_image_gcs_uri,
                                     user_manual_generation_model_flavor,
                                     image_folder_nm,
                                     storage_client)





---



---

### 1.7. Create helper function to render images from Google Cloud Storage in this notebook

This is a helpful utility function that you'll use later in the tutorial. It takes the results of your search query (stored in a pandas DataFrame) and displays the corresponding product images in a nice grid format, making it easy to see how well your search worked.

In [ ]:
# Imports for GCS, plotting, and image handling
from google.cloud import storage
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import io
import math

# Create Cloud Storage client
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(MULTIMODAL_STAGE_BUCKET)

def display_product_images(df):
    """
    Displays product images from a DataFrame in a 3-column grid.
    Simplified for clarity in a tutorial context.
    """

    # Calculate grid size
    num_images = len(df)
    cols = 3
    rows = math.ceil(num_images / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(10, rows * 3.5))

    # Flatten the axes array for easy iteration, regardless of shape
    axs = axs.flatten()

    for i, row in df.iterrows():
        # Extract the image path from the full GCS URI
        image_path = row['product_image_gcs_uri'].split('/', 3)[-1]

        # Get the image from GCS
        blob = bucket.blob(image_path)
        image = Image.open(io.BytesIO(blob.download_as_bytes()))

        # Plot the image
        axs[i].imshow(image)
        axs[i].set_title(image_path.split('/')[-1], fontsize=10)
        axs[i].axis('off')

    # Hide any unused subplots
    for j in range(num_images, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

---

## 2. Prepare sample data

### 2.1. BigQuery dataset


We will reuse the BigQuery dataset we created the previous module called `rscw_multimodal_ai_stg_ds`.

### 2.2. Products table

We will use the table we created in the previous module - `rscw_multimodal_ai_stg_ds.product_master_augmented` that already has some data.

In [ ]:
%%bigquery --project {PROJECT_ID}

select * from rscw_multimodal_ai_stg_ds.product_master_augmented limit 10

### 2.3. Product images

We will generate images for the 10 home appliances that are in `rscw_multimodal_ai_stg_ds.product_master_augmented` and persist them to Google Cloud Storage bucket - `MULTIMODAL_STAGE_BUCKET` in the directory `generated_images`

In [ ]:
import pandas_gbq

model_flavor = "gemini-2.5-flash-image"
image_folder_nm = "generated_images"
genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
storage_client = storage.Client(project=PROJECT_ID)

# Construct the SQL query to select all columns from the specified table
sql = f"SELECT product_id, product_nm, product_description FROM `{PROJECT_ID}.{MULTIMODAL_STAGE_DS}.product_master_augmented`"

# Read the BigQuery table into a pandas DataFrame
df_products = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, dialect="standard")

# Iterate over each row in the DataFrame and call the function
for index, row in df_products.iterrows():
    product_id = row['product_id']
    product_nm = row['product_nm']
    product_description = row['product_description']

    print(f"Generating image for product_id: {product_id}, product_nm: {product_nm}")
    generate_and_persist_image_for_product(
        genai_client,
        storage_client,
        model_flavor,
        product_id,
        product_nm,
        product_description,
        MULTIMODAL_STAGE_BUCKET,
        image_folder_nm
    )

    print("---")
    print(".......................................")
    print("---")

### 2.4. Product user manuals

We will generate user manuals for the 10 home appliances that are in `rscw_multimodal_ai_stg_ds.product_master_augmented` and persist them to Google Cloud Storage bucket - `MULTIMODAL_STAGE_BUCKET` in the directory `generated_user_manuals`

In [ ]:
model_flavor = "gemini-2.5-pro"
image_folder_nm = "generated_images"

#vertexai.init(project=PROJECT_ID, location=LOCATION)
genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
storage_client = storage.Client(project=PROJECT_ID)

# Construct the SQL query to select all columns from the specified table
sql = f"SELECT product_id, product_nm, product_description FROM `{PROJECT_ID}.{MULTIMODAL_STAGE_DS}.product_master_augmented`"

# Read the BigQuery table into a pandas DataFrame
df_products = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, dialect="standard")

# Iterate over each row in the DataFrame and call the function
for index, row in df_products.iterrows():
    product_id = row['product_id']
    product_nm = row['product_nm']
    product_description = row['product_description']
    product_image_gcs_uri=f"gs://{MULTIMODAL_STAGE_BUCKET}/generated_images/{product_id}.png"

    print(f"Generating user manual for product_id: {product_id}, product_nm: {product_nm}")

    generate_and_persist_user_manual_pdf(product_id,
                                     product_nm,
                                     product_description,
                                     product_image_gcs_uri,
                                     model_flavor,
                                     image_folder_nm,
                                     storage_client)

    print("---")
    print("....................................... ")
    print("---")

### 2.4. Quick review of the product data

Lets update the table with the image URI and user manual content we generated



In [ ]:
from google.cloud import bigquery

# Initialize a BigQuery client
client = bigquery.Client()

# Define your dataset and table IDs
dataset_id = f"{PROJECT_ID}.{MULTIMODAL_STAGE_DS}"
table_id = "product_master_augmented"

# Construct the DML UPDATE statement
# Replace 'your_project_id', 'your_dataset_id', 'your_table_id'
# and the SET/WHERE clauses with your specific requirements.
dml_statement = f"""
    UPDATE {dataset_id}.{table_id}
    SET
    product_image_gcs_uri = CONCAT("gs://{MULTIMODAL_STAGE_BUCKET}","/generated_images/",product_id,".png"),
    product_user_manual_gcs_uri = CONCAT("gs://{MULTIMODAL_STAGE_BUCKET}","/generated_user_manuals/",product_id,".pdf")
    WHERE TRUE;
"""

# Execute the query
query_job = client.query(dml_statement)

# Wait for the job to complete
query_job.result()

print(f"Table {dataset_id}.{table_id} updated successfully.")

Run a quick `SELECT` query to view a couple of rows and confirm that your `products` table has all that we need for the tutorial

In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT *
FROM rscw_multimodal_ai_stg_ds.product_master_augmented
LIMIT 2



---


## 3. Perform text-to-text semantic search

With the product data prepared, it's time to perform the first [semantic search](https://cloud.google.com/discover/what-is-semantic-search). You want to input a text query, like "kitten toy" and find the most similar products. To do this, you first need to convert the product names and descriptions into [vector embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings) -- numerical representations that capture their semantic meaning. Once the embeddings are generated, you can run a vector search.


### 3.1. Create remote model for text embedding

Begin by creating a remote model in BigQuery named `text_embedding_model`. This command doesn't create a new model; instead, it creates a pointer to an existing, pre-trained text embedding model hosted on Vertex AI ([`gemini-embedding-001`](https://ai.google.dev/gemini-api/docs/embeddings#model-versions)). This allows you to leverage Vertex AI models directly from your SQL queries.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL `rscw_multimodal_ai_stg_ds.text_embedding_model`
  REMOTE WITH CONNECTION `us-central1.rscw_cloud_resource_connection_usc`
  OPTIONS (ENDPOINT = 'gemini-embedding-001');

### 3.2. Generate text embeddings for product descriptions

 To store embeddings, alter the `products` table to add a new column. Since embeddings are numerical representations of text (an array of floating point numbers), the column type will be `ARRAY<FLOAT>`.

In [ ]:
%%bigquery --project {PROJECT_ID}

ALTER TABLE rscw_multimodal_ai_stg_ds.product_master_augmented
ADD COLUMN text_embedding ARRAY<FLOAT64>;

Now, generate embeddings with the [`ML.GENERATE_EMBEDDING`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-embedding) function. The following query passes the `product_nm` and `product_description` to the `text_embedding_model` for each item. An `UPDATE` statement saves the resulting embeddings in a new column.

In [ ]:
%%bigquery --project {PROJECT_ID}

UPDATE `rscw_multimodal_ai_stg_ds.product_master_augmented` AS t
SET t.text_embedding = s.ml_generate_embedding_result
FROM (
  SELECT
    product_id,
    ml_generate_embedding_result
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `rscw_multimodal_ai_stg_ds.text_embedding_model`,
      (
        SELECT
          product_id,
          CONCAT(product_nm, ' ', product_description) AS content
        FROM `rscw_multimodal_ai_stg_ds.product_master_augmented`
      ),
      STRUCT(TRUE AS flatten_json_output)
    )
) AS s
WHERE t.product_id = s.product_id;

You should now see the `text_embedding` column populated with a long array of numbers. Those are vector embeddings that numerically represent each products' name and description!

In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT product_id, product_nm, product_description, text_embedding
FROM rscw_multimodal_ai_stg_ds.product_master_augmented
LIMIT 2

### 3.3. Use `VECTOR SEARCH` to perform a text-to-text search

Now, use [`VECTOR_SEARCH`](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search) to perform a text-to-text search. This function works in two main steps:
* A subquery generates an embedding for a search query (e.g. "fabric steamer")
* `VECTOR_SEARCH` then compares this new embedding against all existing product embeddings and returns the `top_k` (3 in this case) most similar results

In [ ]:
%%bigquery text_matches --project {PROJECT_ID}

SELECT base.product_id, base.product_nm, base.product_description, base.product_image_gcs_uri,distance
    FROM
      VECTOR_SEARCH(
        TABLE `rscw_multimodal_ai_stg_ds.product_master_augmented`,
        'text_embedding',
        (
        -- GENERATE AN EMBEDDING AS A SUBQUERY
        SELECT
          ml_generate_embedding_result,
          content AS query
        FROM
          ML.GENERATE_EMBEDDING(
            MODEL `rscw_multimodal_ai_stg_ds.text_embedding_model`,
            ( SELECT "fabric steamer" AS content)
          )
        ),
        top_k => 3)
ORDER BY distance ASC;

The results of the vector search are now stored in a pandas DataFrame called `text_matches` (a parameter added to the [`%%bigquery` magic](https://cloud.google.com/python/docs/reference/bigquery/latest/magics) utility in the prior cell).

Let's display the top rows.

In [ ]:
text_matches.head()

### 3.4. View search results
Now you can use the helper function to display the images of the top 3 products the vector search returned. As you can see, the results are semantically related to "fabric steamer" even if the exact words don't appear in the product names or descriptions.

In [ ]:
display_product_images(text_matches)



---


## 4. Perform text-to-image (multimodal) semantic search

The previous example performed a text-to-text search. Now we can look at multimodal search. This allows you to use a text query to find products based on their visual characteristics (not using any written descriptions). Note that multimodal search also extends to audio, video, and more.

### 4.1. Create remote model for multimodal embedding

To work with images, you'll need a different embedding model. You'll now create a remote model pointing to [`multimodalembedding@001`](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-multimodal-embeddings#supported-models). Unlike the previous text-only model, this one is **multimodal**, meaning it can generate a single vector embedding from content that's text, an image, a video, audio, or more.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL `rscw_multimodal_ai_stg_ds.mm_embedding_model`
  REMOTE WITH CONNECTION `us-central1.rscw_cloud_resource_connection_usc`
  OPTIONS (ENDPOINT = 'multimodalembedding@001');

### 4.2. Generate multimodal embeddings for product images

You need a new column in your table to store the embeddings generated from the product images.

In [ ]:
%%bigquery --project {PROJECT_ID}

ALTER TABLE rscw_multimodal_ai_stg_ds.product_master_augmented
ADD COLUMN mm_embedding ARRAY<FLOAT64>;

This step generates embeddings directly from product images. The process again uses [`ML.GENERATE_EMBEDDING`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-embedding), but with the `mm_embedding_model`.

The key difference is how you provide the image to the model. Instead of a text string, the model requires a temporary, secure URL for each image. Because we haven't stored [`ObjectRef`](https://cloud.google.com/bigquery/docs/reference/standard-sql/objectref_functions)s in our table, we can generate them on the fly by chaining three functions together:

* [`OBJ.MAKE_REF`](https://cloud.google.com/bigquery/docs/reference/standard-sql/objectref_functions#objmake_ref): First, it converts the GCS path from the `uri` column into a structured `ObjectRef`
* [`OBJ.FETCH_METADATA`](https://cloud.google.com/bigquery/docs/reference/standard-sql/objectref_functions#objfetch_metadata): Next, it populates the `ObjectRef` with file metadata from Cloud Storage
* [`OBJ.GET_ACCESS_URL`](https://cloud.google.com/bigquery/docs/reference/standard-sql/objectref_functions#objget_access_url): Finally, it generates a signed URL from the metadata-rich `ObjectRef` that the model can access.

The resulting vector embedding for each image is then saved to the `mm_embedding` column with an `UPDATE` statement.

In [ ]:
%%bigquery --project {PROJECT_ID}

UPDATE `rscw_multimodal_ai_stg_ds.product_master_augmented` AS t
SET
  t.mm_embedding = s.ml_generate_embedding_result
FROM (
  SELECT
    product_id,
    ml_generate_embedding_result
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `rscw_multimodal_ai_stg_ds.mm_embedding_model`,
      (
        SELECT
          product_id,
          OBJ.GET_ACCESS_URL(OBJ.FETCH_METADATA(OBJ.MAKE_REF(product_image_gcs_uri, 'us-central1.rscw_cloud_resource_connection_usc')), 'r') AS content
        FROM
          `rscw_multimodal_ai_stg_ds.product_master_augmented`
      ),
      STRUCT(TRUE AS flatten_json_output)
    )
) AS s
WHERE t.product_id = s.product_id;

 Inspect the table one more time to see your new `mm_embedding` column filled with multimodal embeddings, derived from product images.

In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT product_id, product_nm, product_description, mm_embedding
FROM rscw_multimodal_ai_stg_ds.product_master_augmented
LIMIT 2

### 4.3. Use `VECTOR SEARCH` to perform a text-to-image search

The power of a multimodal model is that it places both text and images into the same "embedding space." This allows you to use an embedding generated from a text string (like "fabric steamer") and directly compare it against embeddings generated from images.

The process for this text-to-image search mirrors the previous text-to-text search, with two key differences:
* A text query ("fabric steamer") is embedded using the multimodal `mm_embedding_model`
* The search is performed against the image embedding column (`embedding_mm_image`) to find the most visually similar items

In [ ]:
%%bigquery multimodal_matches --project {PROJECT_ID}

SELECT base.product_id, base.product_nm, base.product_description, base.product_image_gcs_uri, distance
    FROM
      VECTOR_SEARCH(
        TABLE`rscw_multimodal_ai_stg_ds.product_master_augmented`,
        'mm_embedding',
        (
        SELECT
          ml_generate_embedding_result,
          content AS query
        FROM
          ML.GENERATE_EMBEDDING(
            MODEL `rscw_multimodal_ai_stg_ds.mm_embedding_model`,
            (SELECT "fabric steamer" AS content)
          )
        ),
        'ml_generate_embedding_result',
        top_k => 3)
ORDER BY distance ASC;

### 4.4. View search results

Finally, let's visualize the results of the text-to-image search. You should see images of products that are conceptually related to "kitten toy," demonstrating the power of multimodal embeddings to find relevant items based on visual similarity to a text description.

In [ ]:
# Display the results
display_product_images(multimodal_matches)

---
# 5. BigQuery Automated Embedding Generation

BigQuery has a new feature for automated embedding generation.
`CREATE TABLE` syntax, `GENERATED ALWAYS AS AI.EMBED(...)`, automatically generates embeddings for new records, ensuring data is always AI-ready.

### 5.1. Create table configured for automated embedding generation



In [ ]:
%%bigquery
CREATE TABLE rscw_multimodal_ai_stg_ds.product_master_auto_embeddings (
  product_id STRING,
  product_nm STRING,
  product_description STRING,
  product_image_gcs_uri STRING,
  product_user_manual_gcs_uri STRING,
  description_embedding STRUCT<result ARRAY<FLOAT64>, status STRING>
  GENERATED ALWAYS AS (AI.EMBED(
      product_description,
      connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
      endpoint => 'text-embedding-005'
    ))
    STORED OPTIONS( asynchronous = TRUE )

);


### 5.2. Insert a few records

In [ ]:
%%bigquery
INSERT INTO rscw_multimodal_ai_stg_ds.product_master_auto_embeddings
(product_id, product_nm, product_description, product_image_gcs_uri, product_user_manual_gcs_uri)
select product_id, product_nm, product_description, product_image_gcs_uri, product_user_manual_gcs_uri
from rscw_multimodal_ai_stg_ds.product_master_augmented;

### 5.3. Review the embeddings generated

In [ ]:
%%bigquery
SELECT * FROM rscw_multimodal_ai_stg_ds.product_master_auto_embeddings
LIMIT 2

### 5.4. Search with SQL syntax with `AI.SEARCH`

In [ ]:
%%bigquery

SELECT base.product_nm, base.product_description, distance
FROM AI.SEARCH(TABLE rscw_multimodal_ai_stg_ds.product_master_auto_embeddings, 'product_description', "A whisk to use in the kitchen")
order by distance
limit 3

---

#### This concludes the module, return to the user manual.